In [1]:
import numpy as np
import pandas as pd 
import pyomo.environ as pyo

In [2]:
lojas = pd.read_excel('./Estoque_Ruptura.xlsx', sheet_name='Lojas')
disp_sku = pd.read_excel('./Estoque_Ruptura.xlsx', sheet_name='Disponibilidade_SKU')
lojas_sku = pd.read_excel('./Estoque_Ruptura.xlsx', sheet_name='Loja_SKU')
prob_rupt = pd.read_excel('./Estoque_Ruptura.xlsx', sheet_name='Prob_Rup')

In [3]:
disp_sku
lojas_sku
prob_rupt

,Loja,SKU,NIVEL_RUPTURA,QUANTIDADE_STOCADA,PROBABILIDADE_RUPTURA
0,ASSAI,Mimmo Plus Suave,1,5,92
1,ASSAI,Mimmo Plus Suave,2,13,81
2,ASSAI,Mimmo Plus Suave,3,26,62
3,ASSAI,Mimmo Plus Suave,4,39,32
4,ASSAI,Mimmo Plus Suave,5,50,22
...,...,...,...,...,...
79,WALLMART,Report Premium,2,10,84
80,WALLMART,Report Premium,3,21,69
81,WALLMART,Report Premium,4,31,47
82,WALLMART,Report Premium,5,39,22


In [4]:
model = pyo.ConcreteModel()

### Conjuntos

In [5]:
list_lojas = lojas.Lojas.unique()
model.lojas = pyo.Set(initialize=list_lojas, doc="Lista de lojas")

In [6]:
list_sku = disp_sku.SKU.unique()
model.sku = pyo.Set(initialize=list_sku, doc="Lista de SKU's")

In [7]:
#list_lojasXsku = 
list_lojasXsku = [(row['Loja'], row["SKU"]) for indx, row in lojas_sku.iterrows()]
list_lojasXsku
model.lojXsku = pyo.Set(initialize=list_lojasXsku, doc="Lojas por SKU")

In [8]:
#niv_rupt 
list_niv_rupt = prob_rupt.NIVEL_RUPTURA.unique()
list_niv_rupt
model.niv_rup = pyo.Set(initialize=list_niv_rupt, doc="Níveis de Ruptura")

In [9]:
list_lojaXskuXniv_rupt = [(row['Loja'], row['SKU'], row['NIVEL_RUPTURA']) for _, row in prob_rupt.iterrows()]
list_lojaXskuXniv_rupt
model.LSKUR = pyo.Set(initialize=list_lojaXskuXniv_rupt, doc="Lojas X SKU X Níve de Ruptura")

### Parâmetros

In [10]:
disp_sku.to_dict()

{'SKU': {0: 'Mimmo Plus Suave', 1: 'Report Premium'},
 'Quantidade disponivel': {0: 250, 1: 150}}

In [11]:
def df_to_dict(df, indx_col, value_col):
    all_cols = indx_col.copy()
    all_cols.append(value_col)

    return df[all_cols].set_index(indx_col).to_dict()[value_col]

In [12]:
disp_dict = df_to_dict(df=disp_sku, indx_col=['SKU'], value_col="Quantidade disponivel")
disp_dict

{'Mimmo Plus Suave': 250, 'Report Premium': 150}

In [13]:
model.sku.pprint()

sku : Lista de SKU's
    Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    2 : {'Mimmo Plus Suave', 'Report Premium'}


In [14]:
model.disp = pyo.Param(
    model.sku,
    initialize=disp_dict,
    within=pyo.NonNegativeReals,
    doc="disponibilidade de SKU's"    
)

In [15]:
lojas_sku.columns

Index(['Loja', 'SKU', 'Capacidade Maxima', 'Probabilidade Aceitavel'], dtype='object')

In [16]:
lojas_sku_dict = df_to_dict(df=lojas_sku, indx_col=['Loja','SKU'], value_col="Capacidade Maxima")

In [17]:
lojas_sku_dict

{('ASSAI', 'Mimmo Plus Suave'): 53,
 ('EXTRA', 'Mimmo Plus Suave'): 80,
 ('G. BARBOSA', 'Mimmo Plus Suave'): 20,
 ('MERCANTIL RODRIGUES', 'Mimmo Plus Suave'): 55,
 ('PAO DE AÇUCAR', 'Mimmo Plus Suave'): 40,
 ('TODO DIA', 'Mimmo Plus Suave'): 33,
 ('WALLMART', 'Mimmo Plus Suave'): 82,
 ('ASSAI', 'Report Premium'): 37,
 ('BOM PRECIO', 'Report Premium'): 37,
 ('EXTRA', 'Report Premium'): 52,
 ('G. BARBOSA', 'Report Premium'): 53,
 ('MERCANTIL RODRIGUES', 'Report Premium'): 29,
 ('PAO DE AÇUCAR', 'Report Premium'): 24,
 ('WALLMART', 'Report Premium'): 42}

In [18]:
model.CapMaxLS = pyo.Param(
    model.lojas,
    model.sku,
    initialize=lojas_sku_dict,
    within=pyo.NonNegativeReals,
    doc="Capacidade máxima da loja"
)

In [19]:
lojas_sku
prob_aceitavel_dict = df_to_dict(df=lojas_sku, indx_col=['Loja','SKU'], value_col="Probabilidade Aceitavel")

In [20]:
model.Prob_actv = pyo.Param(
    model.lojas,
    model.sku,
    initialize=prob_aceitavel_dict,
    within=pyo.NonNegativeReals,
    doc="Probabilidade Aceitável por Loja"
)

In [21]:
prob_rupt
quant_stq_dict = df_to_dict(df=prob_rupt, indx_col=['Loja','SKU','NIVEL_RUPTURA'], value_col='QUANTIDADE_STOCADA')

In [22]:
model.Qstq = pyo.Param(
    model.LSKUR,
    initialize=quant_stq_dict,
    within=pyo.NonNegativeReals,
    doc="Quantidade Estocada na loja"
)

In [23]:
prob_rupt_dict = df_to_dict(df=prob_rupt, indx_col=['Loja','SKU','NIVEL_RUPTURA'], value_col='PROBABILIDADE_RUPTURA')
#prob_rupt_dict

In [24]:
model.Prob_rupt = pyo.Param(
    model.LSKUR,
    initialize=prob_rupt_dict,
    within=pyo.NonNegativeReals,
    doc="Probabilidade de Ruptura"
)

### Variaveis

In [25]:
lojas_sku_dict

{('ASSAI', 'Mimmo Plus Suave'): 53,
 ('EXTRA', 'Mimmo Plus Suave'): 80,
 ('G. BARBOSA', 'Mimmo Plus Suave'): 20,
 ('MERCANTIL RODRIGUES', 'Mimmo Plus Suave'): 55,
 ('PAO DE AÇUCAR', 'Mimmo Plus Suave'): 40,
 ('TODO DIA', 'Mimmo Plus Suave'): 33,
 ('WALLMART', 'Mimmo Plus Suave'): 82,
 ('ASSAI', 'Report Premium'): 37,
 ('BOM PRECIO', 'Report Premium'): 37,
 ('EXTRA', 'Report Premium'): 52,
 ('G. BARBOSA', 'Report Premium'): 53,
 ('MERCANTIL RODRIGUES', 'Report Premium'): 29,
 ('PAO DE AÇUCAR', 'Report Premium'): 24,
 ('WALLMART', 'Report Premium'): 42}

In [26]:
dict_stq = {}
for key, value in lojas_sku_dict.items():
    dict_stq[key] = (0,value)

In [27]:
dict_stq

{('ASSAI', 'Mimmo Plus Suave'): (0, 53),
 ('EXTRA', 'Mimmo Plus Suave'): (0, 80),
 ('G. BARBOSA', 'Mimmo Plus Suave'): (0, 20),
 ('MERCANTIL RODRIGUES', 'Mimmo Plus Suave'): (0, 55),
 ('PAO DE AÇUCAR', 'Mimmo Plus Suave'): (0, 40),
 ('TODO DIA', 'Mimmo Plus Suave'): (0, 33),
 ('WALLMART', 'Mimmo Plus Suave'): (0, 82),
 ('ASSAI', 'Report Premium'): (0, 37),
 ('BOM PRECIO', 'Report Premium'): (0, 37),
 ('EXTRA', 'Report Premium'): (0, 52),
 ('G. BARBOSA', 'Report Premium'): (0, 53),
 ('MERCANTIL RODRIGUES', 'Report Premium'): (0, 29),
 ('PAO DE AÇUCAR', 'Report Premium'): (0, 24),
 ('WALLMART', 'Report Premium'): (0, 42)}

In [28]:
model.Q_STQ = pyo.Var(
    model.lojXsku,
    within=pyo.NonNegativeReals,
    bounds=dict_stq,
    doc="Variável real, indica a quantidade do estoque"
)

In [29]:
model.Q_STQ.pprint()

Q_STQ : Variável real, indica a quantidade do estoque
    Size=14, Index=lojXsku
    Key                                         : Lower : Value : Upper : Fixed : Stale : Domain
                  ('ASSAI', 'Mimmo Plus Suave') :     0 :  None :    53 : False :  True : NonNegativeReals
                    ('ASSAI', 'Report Premium') :     0 :  None :    37 : False :  True : NonNegativeReals
               ('BOM PRECIO', 'Report Premium') :     0 :  None :    37 : False :  True : NonNegativeReals
                  ('EXTRA', 'Mimmo Plus Suave') :     0 :  None :    80 : False :  True : NonNegativeReals
                    ('EXTRA', 'Report Premium') :     0 :  None :    52 : False :  True : NonNegativeReals
             ('G. BARBOSA', 'Mimmo Plus Suave') :     0 :  None :    20 : False :  True : NonNegativeReals
               ('G. BARBOSA', 'Report Premium') :     0 :  None :    53 : False :  True : NonNegativeReals
    ('MERCANTIL RODRIGUES', 'Mimmo Plus Suave') :     0 :  None :    55 :

In [30]:
model.RUPT_SKU = pyo.Var(
    model.lojXsku,
    within=pyo.NonNegativeReals,
    doc="Variável real, indica o nível de ruptura do sku"
)

In [31]:
model.RUPT_L = pyo.Var(
    model.lojas,
    within=pyo.Binary,
    doc="Variável binaria, indica se houve ruptura na loja"
)

In [32]:
model.RUPT_L_SKU = pyo.Var(
    model.lojXsku,
    within=pyo.Binary,
    doc="Variável real, indica se houve ruptura do produto na loja"
)

In [33]:
model.NIVEL_SKU = pyo.Var(
    model.LSKUR,
    within=pyo.Binary,
    doc="Variável binaria, indica o nível do stoque"
)

In [34]:
model.LAMBDA = pyo.Var(
    model.LSKUR,
    within=pyo.Binary,
    doc="Função auxiliar na linearização por partes"
)

### Restrições

In [35]:
def _estoque_max(model, l, s):
    eq = (sum(
            model.NIVEL_SKU[(l,s,r)]*model.Qstq[(l,s,r)] for r in model.niv_rup 
        ) <= model.CapMaxLS[(l,s)]
    )

    return eq

model.rest_estoque_maximo = pyo.Constraint(model.lojXsku, rule=_estoque_max)

In [36]:
def _quant_sku(model, l, s):
    eq = (
        sum(
            model.LAMBDA[(l, s, r)]*model.Qstq[(l, s, r)] for r in model.niv_rup
        ) == model.Q_STQ[(l,s)]
    )

    return eq

model.rest_quant_sku = pyo.Constraint(model.lojXsku, rule=_quant_sku)

In [37]:
def _rupt_sku(model, l, s):
    eq = (
        sum(
            model.LAMBDA[(l,s,r)]*model.Prob_rupt[(l,s,r)] for r in model.niv_rup
        ) == model.RUPT_SKU[(l, s)]
    )

    return eq

model.rest_rupt_sku = pyo.Constraint(model.lojXsku, rule=_rupt_sku)

In [38]:
def _lim_disp(model, s):
    eq = (
        sum(
            model.Q_STQ[(l,s_)] for l, s_ in model.lojXsku if s_==s
        ) <= model.disp[(s)]
    )

    return eq

model.rest_lim_disp = pyo.Constraint(model.sku, rule=_lim_disp)

In [39]:
def _max_nivel(model, l, s):
    eq = (
        sum(
            model.NIVEL_SKU[(l,s,r)] for r in model.niv_rup
        ) == 1
    )

    return eq

model.rest_max_nivel = pyo.Constraint(model.lojXsku, rule=_max_nivel)

In [40]:
def _max_lambda(model, l, s):
    eq = (
        sum(
            model.LAMBDA[(l,s,r)] for r in model.niv_rup
        ) == 1
    )

    return eq

model.rest_max_lambda = pyo.Constraint(model.lojXsku, rule=_max_lambda)

In [41]:
def _min_nivel(model, l, s, r):
    if r == 1:
        eq = (
            model.LAMBDA[(l,s,r)] <= model.NIVEL_SKU[(l,s,r)]
        )
    elif r == 6:
        eq = (
            model.LAMBDA[(l,s,r)] <= model.NIVEL_SKU[(l,s,r-1)]
        )
    else:
        eq = (
            model.LAMBDA[(l,s,r)] <= model.NIVEL_SKU[(l,s,r)] + model.NIVEL_SKU[(l,s,r-1)]
        )
    
    return eq

model.rest_min_nivel = pyo.Constraint(model.LSKUR, rule=_min_nivel)

In [42]:
def _houve_ruptura_sku(model, l, s):
    eq = (
        model.RUPT_SKU[(l,s)] - model.Prob_actv[(l,s)] <= model.RUPT_L_SKU[(l,s)]
    )

    return eq

model.rest_houve_rupt_sku = pyo.Constraint(model.lojXsku, rule=_houve_ruptura_sku)

In [43]:
def _houve_rupt_loja(model, l, s):
    eq = model.RUPT_L[l] >= model.RUPT_L_SKU[(l,s)]

    return eq

model.rest_houve_rupt_loja = pyo.Constraint(model.lojXsku,rule=_houve_rupt_loja)

### Função Objetivo

In [44]:
def _fo(model):
    value = sum(model.RUPT_L_SKU[(l,s)] for l, s in model.lojXsku)
    
    return value


model.fo = pyo.Objective(rule=_fo, sense=pyo.minimize, doc="Estoque_Ruptura")

In [45]:
model.write("modelo_stoque_ruptura.lp", io_options={'symbolic_solver_labels':True})

('modelo_stoque_ruptura.lp', 2392455663776)

In [46]:
path_solver = '../../solvers/cbc.exe'
solver = pyo.SolverFactory('cbc', executable=path_solver)
sol = solver.solve(model, tee=False)

model.name="unknown";
    - termination condition: infeasible
    - message from solver: <undefined>


In [47]:
sol.write(num=1)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 156
  Number of variables: 196
  Number of binary variables: 190
  Number of integer variables: 190
  Number of nonzeros: 14
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.01
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
     